In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2025-06-07 21:35:42--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-06-07 21:35:42 (63.9 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [10]:
import minsearch

In [11]:
import json

In [12]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [14]:
#docs_raw

In [13]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [14]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Creating an INDEX(INDEXING)


`keyword_fields` are in elasticsearch an analogous approach to the sentence `SELECT * WHERE course = 'data-engineering-zoomcamp'` this example is considering the `documents[0]` document from the `documents.json` file. 

Obs: `keyword_fields` are for filtering and `text_fields` are for performance search

In [40]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [16]:
q = 'the course has already started, can I still enroll?'

## 'fit' Index, an analogous to the `.fit` method from scikit-learn library

Adjusting the search to the knowledge base (documents) instantiation

In [17]:
Index.fit(documents)

In [18]:
boost = {'question': 3.0, 'section': 0.5}

results = Index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5 
)

In [19]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Generation


In [20]:
from openai import OpenAI

In [21]:
client = OpenAI()

In [22]:
q

'the course has already started, can I still enroll?'

In [23]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [24]:
response.choices[0].message.content


"Whether you can still enroll in a course that has already started depends on several factors, including the policies of the institution or platform offering the course, the nature of the course, and its current progress. Here are some steps you can take:\n\n1. **Contact the Institution or Instructor:** Reach out to the course administrator or instructor to inquire about late enrollment policies. They can provide the most accurate information regarding your situation.\n\n2. **Check the Institution's Policies:** Review the enrollment policies on the institution’s website. Some programs may allow late entry within a certain timeframe or have a waitlist.\n\n3. **Assess the Course Content:** If the course is self-paced or offered online, it may be easier to catch up on missed material. However, for in-person or synchronous courses, consider the amount of work required to catch up.\n\n4. **Consider the Impact:** Think about how enrolling late will affect your workload and whether you’ll hav

### Prompting

Feeding the context to the LLM!

It's a good practice to give the model a rol
* (Usually we need to do this for advanced LLMs)
* (at the moment will be using an trial on error approach)
* (Late in the course metrics will help us to validate the efficiency of the prompts)

In [26]:
prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the question 
    If the CONTEXT doesn't contain the answer, output NONE

    QUESTION: {question}

    CONTEXT: 
    {context}
""".strip()

In [22]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

#### LET'S BUILD THE CONTEXT

In [29]:
context = ""

for doc in results: 
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [32]:
prompt = prompt_template.format(question=q, context=context).strip()

In [33]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the question 
    If the CONTEXT doesn't contain the answer, output NONE

    QUESTION: the course has already started, can I still enroll?

    CONTEXT: 
    section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also sta

In [34]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content



"Yes, even if you don't register before the course start date, you're still eligible to submit the homework. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

## CONCLUSION

We search/retrieve our results with `minsearch` wrapped up as a context and feed it to the LLM through the `prompt`

## CLEAN CODE

In [41]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [42]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [43]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [44]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [48]:
query = 'how do I run kafka?'

def rag(query):
    
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [49]:
rag(query)

"To run Kafka, you need to execute your Kafka-related Java classes from the terminal. Here is a guide to running a Java Kafka producer, for example:\n\nIn the project directory, use this command:\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you're working with Python instead of Java, ensure you have set up a virtual environment and have the necessary dependencies installed before running your Python Kafka scripts."

In [50]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. You are eligible to submit homework. However, please note that there will be deadlines for submitting the final projects, so it's advisable not to leave everything until the last minute."

```
    docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```